In [1]:
from gazpacho import get, Soup
import pandas as pd

from datetime import timedelta, date

import time 
import tqdm

In [24]:
dates = 0
def scrape_page(dates=dates):
    url = 'http://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2015-11-3&EndDate=2021-03-04&ILChkBx=yes&InjuriesChkBx=yes&PersonalChkBx=yes&Submit=Search&start=' + str(dates) 
    html = get(url)
    soup = Soup(html)
    
    trs = soup.find('tr')
    tr_pop = trs.pop(0)
    tr = trs[0]
    
    def parse_tr(tr):
        return {
            'date': tr.find('td')[0].text,
            'team': tr.find('td')[1].text,
            'acquired': tr.find('td')[2].text,
            'relinquished': tr.find('td')[3].text,
            'description': tr.find('td')[4].text,
        }
    
    
    rankers = [parse_tr(tr) for tr in trs]
    return rankers

data = []
for i in range(0, 15650, 25):
    data.extend(scrape_page(dates=i))
    time.sleep(.3)

In [27]:
df = pd.DataFrame(data)

In [33]:
df.to_csv("injured_player_reports.csv")

In [31]:
df.drop(df.tail(1).index,inplace=True) 

In [32]:
df

,date,team,acquired,relinquished,description
0,2015-11-03,76ers,,• Robert Covington,sprained MCL in right knee (DTD)
1,2015-11-03,Cavaliers,,• J.R. Smith,bruised right knee (DTD)
2,2015-11-03,Hawks,,• Mike Muscala,sore right ankle (DTD)
3,2015-11-03,Hawks,,• Thabo Sefolosha,placed on IL with ankle injury
4,2015-11-03,Hawks,• Tim Hardaway Jr.,,activated from IL
...,...,...,...,...,...
16270,2021-03-03,Cavaliers,• Taurean Prince,,returned to lineup
16271,2021-03-03,Grizzlies,• Kyle Anderson,,activated from IL
16272,2021-03-03,Hawks,,• Clint Capela,placed on IL with right foot injury
16273,2021-03-03,Hornets,,• P.J. Washington,sprained right ankle (DTD)


In [74]:
df = pd.read_csv("injured_player_reports.csv")
del df["Unnamed: 0"]

In [75]:
df.head(1)

,date,team,acquired,relinquished,description
0,2015-11-03,76ers,NaN,• Robert Covington,sprained MCL in right knee (DTD)


In [77]:
df = df.fillna(0)

In [78]:
z = (df["acquired"] !=  0)

In [79]:
df["added_player"] = 0

In [80]:
df["added_player"][z] = 1

/Users/brandonanhorn19/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [81]:
df["added_player"].value_counts()

0    8995
1    7280
Name: added_player, dtype: int64

In [82]:
y = (df["relinquished"] !=  0)

In [83]:
df["lost_player"] = 0

In [84]:
df["lost_player"][y] = 1

/Users/brandonanhorn19/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [85]:
df

,date,team,acquired,relinquished,description,added_player,lost_player
0,2015-11-03,76ers,0,• Robert Covington,sprained MCL in right knee (DTD),0,1
1,2015-11-03,Cavaliers,0,• J.R. Smith,bruised right knee (DTD),0,1
2,2015-11-03,Hawks,0,• Mike Muscala,sore right ankle (DTD),0,1
3,2015-11-03,Hawks,0,• Thabo Sefolosha,placed on IL with ankle injury,0,1
4,2015-11-03,Hawks,• Tim Hardaway Jr.,0,activated from IL,1,0
...,...,...,...,...,...,...,...
16270,2021-03-03,Cavaliers,• Taurean Prince,0,returned to lineup,1,0
16271,2021-03-03,Grizzlies,• Kyle Anderson,0,activated from IL,1,0
16272,2021-03-03,Hawks,0,• Clint Capela,placed on IL with right foot injury,0,1
16273,2021-03-03,Hornets,0,• P.J. Washington,sprained right ankle (DTD),0,1


In [87]:
prac_df = df[(df["date"] == "2015-11-03")]

In [88]:
prac_df

,date,team,acquired,relinquished,description,added_player,lost_player
0,2015-11-03,76ers,0,• Robert Covington,sprained MCL in right knee (DTD),0,1
1,2015-11-03,Cavaliers,0,• J.R. Smith,bruised right knee (DTD),0,1
2,2015-11-03,Hawks,0,• Mike Muscala,sore right ankle (DTD),0,1
3,2015-11-03,Hawks,0,• Thabo Sefolosha,placed on IL with ankle injury,0,1
4,2015-11-03,Hawks,• Tim Hardaway Jr.,0,activated from IL,1,0
5,2015-11-03,Heat,0,• Amare Stoudemire / Amar'e Stoudemire,placed on IL with rehab of knee,0,1
6,2015-11-03,Heat,0,• Gerald Green (b. 1986-01-26),placed on IL with illness,0,1
7,2015-11-03,Heat,• Chris Andersen,0,activated from IL,1,0
8,2015-11-03,Heat,• Josh Richardson,0,activated from IL,1,0
9,2015-11-03,Kings,0,• DeMarcus Cousins,placed on IL with strained Achilles,0,1


In [97]:
for i in range(len(prac_df)):
    y = (prac_df["team"][i] == prac_df["team"][i + 1])
    print(y)

False
False
True
True
False
True
True
True
False
True
False
False
True
False
True
False
True
True
False
False
True


KeyError: 22

In [ ]:
# wanna find if a start is missing 
# how many players are missing each game
# how many players come back for each game